# Ejercicio Covid-19

In [1]:
import os
import numpy as np
import pandas as pd

#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore2 = date_daybefore.strftime('%d/%m')

date_today_large = date_today_complete.strftime('%Y-%m-%d')
date_daybefore_large = date_daybefore.strftime('%Y-%m-%d')
date_today_large

'2020-05-13'

## Traer los datos de Datadista

In [2]:
data_ccaa_confirmed_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv", sep = ',', encoding="utf-8")

data_ccaa_deaths_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv", sep = ',', encoding="utf-8")

data_ccaa_recovered_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_altas.csv", sep = ',', encoding="utf-8")

data_spain_datadista = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19.csv")


#Comprobar si están subidos los últimos datos, si no sale error es que hay datos.
data_ccaa_recovered_timeseries[date_today_large]

0      9481
1      3312
2      1038
3      1465
4      1475
5      2054
6      6205
7      7544
8     25326
9       156
10     9268
11     2456
12     7996
13    39984
14      115
15     1861
16     3297
17    14986
18     2804
Name: 2020-05-13, dtype: int64

## RESUMEN: Resumen datos globales

In [3]:
# Función para obtener datos del mundo    
'''
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual
'''

def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"


# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)

''' Cogiendo datos de John Hopkins
# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)
'''

# Para España actual(dataset de datadista)
data_confirmed_spain_actual = int(data_ccaa_confirmed_timeseries[date_today_large].sum())
data_recovered_spain_actual = int(data_ccaa_recovered_timeseries[date_today_large].sum())
data_deaths_spain_actual = int(data_ccaa_deaths_timeseries[date_today_large].sum())


# Para España nuevos casos 24h(dataset de datadista)
data_spain_datadista_trans = data_spain_datadista.set_index('fecha').transpose()
data_spain_datadista_confirmed = data_spain_datadista_trans.iloc[1]
data_spain_datadista_recovered = data_spain_datadista_trans.iloc[3]
data_spain_datadista_deaths = data_spain_datadista_trans.iloc[4]
data_confirmed_spain_24h = int(data_spain_datadista_confirmed[date_today_large]) - int(data_spain_datadista_confirmed[date_daybefore_large])
data_recovered_spain_24h = int(data_spain_datadista_recovered[date_today_large]) - int(data_spain_datadista_recovered[date_daybefore_large])
data_deaths_spain_24h= int(data_spain_datadista_deaths[date_today_large]) - int(data_spain_datadista_deaths[date_daybefore_large])

# Creamos dataframes

data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_recovered_spain_actual, 
                                    'Fallecidos': data_deaths_spain_actual}, 
                                   index=['Casos totales en España'])

data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])


data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

########### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_resume_actual, data_spain_newcases])

data_resume_final.to_csv('data_world_resume.csv', sep = ';')
print(data_resume_final)

Confirmados  Curados  Fallecidos
Casos totales en el mundo           4261747  1493414      291942
Nuevos casos en el mundo (24h)        84245    37205        5612
Casos totales en España              271095   140823       27104
Nuevos casos en España (24h)            439     1843         184


## MUNDO: Preparación general de datos por países

In [4]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-05-08,83976.0,176202.0,170588.0,104691.0,217185.0,10840.0,8070.0,222857.0,1283929.0,212629.0
2020-05-09,83990.0,176782.0,171324.0,106220.0,218268.0,10874.0,8099.0,223578.0,1309550.0,216525.0
2020-05-10,84010.0,177094.0,171879.0,107603.0,219070.0,10909.0,8105.0,224350.0,1329260.0,220449.0
2020-05-11,84011.0,177547.0,172576.0,109286.0,219814.0,10936.0,8132.0,227436.0,1347881.0,224332.0
2020-05-12,84018.0,178349.0,173171.0,110767.0,221216.0,10962.0,8157.0,228030.0,1369376.0,227741.0


## MUNDO: Confirmados, curados y fallecidos

In [5]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together_final = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together_final.to_csv('data_world_actual.csv', sep = ';')
data_together_final

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,84018.0,79222.0,4637.0
France,178349.0,57898.0,26994.0
Germany,173171.0,147200.0,7738.0
Italy,221216.0,109039.0,30911.0
Spain,228030.0,138980.0,26920.0
US,1369376.0,230287.0,82356.0
United Kingdom,227741.0,1023.0,32769.0


## MUNDO: Confirmados, curados y fallecidos por habitantes de población

In [6]:
data_together

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,84018.0,178349.0,173171.0,221216.0,228030.0,1369376.0,227741.0
Curados,79222.0,57898.0,147200.0,109039.0,138980.0,230287.0,1023.0
Fallecidos,4637.0,26994.0,7738.0,30911.0,26920.0,82356.0,32769.0


In [7]:
pob_wuhan = 11895000
pob_china = 1393000000
pob_france = 67028048
pob_germany = 83019200
pob_italy = 60541000
pob_spain = 46940000
pob_US = 328200000
pob_UK = 66650000

# Por cada millón de habitantes hay estos casos...
data_together_den = data_together.copy()
data_together_den['China'] = (data_together_den['China'] / pob_china)*1000000 # Por cada caso infectado hay 142 que no lo están
data_together_den['France'] = (data_together_den['France'] / pob_france)*1000000 # Por cada caso infectado hay 455 que no lo están
data_together_den['Germany'] = (data_together_den['Germany'] / pob_germany)*1000000  # Por cada caso infectado hay 455 que no lo están
data_together_den['Italy'] = (data_together_den['Italy'] / pob_italy)*1000000
data_together_den['Spain'] = (data_together_den['Spain'] / pob_spain)*1000000
data_together_den['US'] = (data_together_den['US'] / pob_US)*1000000
data_together_den['United Kingdom'] = (data_together_den['United Kingdom'] / pob_UK)*1000000
data_together_den = data_together_den.round(0)
# Exporto CSV
data_together_den.to_csv('data_world_actual_den.csv', sep = ';')
data_together_den

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,60.0,2661.0,2086.0,3654.0,4858.0,4172.0,3417.0
Curados,57.0,864.0,1773.0,1801.0,2961.0,702.0,15.0
Fallecidos,3.0,403.0,93.0,511.0,573.0,251.0,492.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [8]:
# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,81)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, 
                                                 dataframe_Iran, dataframe_Italy, dataframe_Korea, 
                                                 dataframe_Norway, dataframe_Spain, dataframe_US,
                                                 dataframe_UnitedKingdom])

data_drive_from100cases.to_csv('data_world_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
76,NaN,82718.0,NaN,NaN,219070.0,10822.0,NaN,NaN,NaN,NaN
77,NaN,82809.0,NaN,NaN,219814.0,10840.0,NaN,NaN,NaN,NaN
78,NaN,82883.0,NaN,NaN,221216.0,10874.0,NaN,NaN,NaN,NaN
79,NaN,82941.0,NaN,NaN,NaN,10909.0,NaN,NaN,NaN,NaN
80,NaN,83014.0,NaN,NaN,NaN,10936.0,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [9]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('data_world_timeline.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-05-08,83976.0,176202.0,170588.0,104691.0,217185.0,10840.0,8070.0,222857.0,1283929.0,212629.0
2020-05-09,83990.0,176782.0,171324.0,106220.0,218268.0,10874.0,8099.0,223578.0,1309550.0,216525.0
2020-05-10,84010.0,177094.0,171879.0,107603.0,219070.0,10909.0,8105.0,224350.0,1329260.0,220449.0
2020-05-11,84011.0,177547.0,172576.0,109286.0,219814.0,10936.0,8132.0,227436.0,1347881.0,224332.0
2020-05-12,84018.0,178349.0,173171.0,110767.0,221216.0,10962.0,8157.0,228030.0,1369376.0,227741.0


## ESPAÑA: Resumen datos España: data_spain_newcases24h.csv

In [10]:
data_spain_datadista

,fecha,casos_total,casos_pcr,casos_test_ac,altas,fallecimientos,ingresos_uci,hospitalizados
0,2020-02-21,2,2,NaN,NaN,NaN,NaN,NaN
1,2020-02-22,2,2,NaN,NaN,NaN,NaN,NaN
2,2020-02-23,2,2,NaN,NaN,NaN,NaN,NaN
3,2020-02-24,2,2,NaN,NaN,NaN,NaN,NaN
4,2020-02-25,3,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
78,2020-05-09,263166,223961,39205.0,133952.0,26478.0,11292.0,122265.0
79,2020-05-10,267650,227377,40273.0,136166.0,26621.0,11328.0,122730.0
80,2020-05-11,268477,227770,40707.0,137139.0,26744.0,11348.0,122971.0
81,2020-05-12,269742,228252,41490.0,138980.0,26920.0,11371.0,123484.0


In [11]:
# Con datos de datadista, SOLO PARA DATOS DE PAÍS
data_path_confirmed = 'casos_total'
data_path_recovered = 'altas'
data_path_deaths = 'fallecimientos'

# Evolución últimos 40 días de nuevos casos

def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = data_spain_datadista.set_index('fecha')
    data = data[data_path]
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

data_day(data_path_confirmed, -3)

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

data_spain_confirmed24h_datadista = create_dataframe(data_day, data_path_confirmed, ['Confirmados (24h) en España'])
data_spain_recovered24h_datadista = create_dataframe(data_day, data_path_recovered, ['Recuperados (24h) en España'])
data_spain_deaths24h_datadista = create_dataframe(data_day, data_path_deaths, ['Fallecidos (24h) en España'])
data_spain_newcases24h_datadista = pd.concat([data_spain_confirmed24h_datadista,data_spain_recovered24h_datadista,data_spain_deaths24h_datadista], axis=0)


# Exportar
data_spain_newcases24h_datadista.to_csv('data_spain_newcases24h.csv', sep = ';')
data_spain_deaths24h_datadista.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')
data_spain_newcases24h_datadista

,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,...,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12
Confirmados (24h) en España,6675,5534,3668,5262,5606,5796,4603,4958,4051,3133,...,999,1975,3000,2967,3025,2688,4484,827,1265,1353
Recuperados (24h) en España,3633,3528,2022,2897,4471,4025,3744,3258,2908,2139,...,2441,2143,2516,2509,2637,2804,2214,973,1841,1843
Fallecidos (24h) en España,780,670,628,757,757,781,642,500,606,551,...,164,185,244,213,229,179,143,123,176,184


In [12]:
#Con datos de John Hopkins, SOLO PARA COMPARAR CON INTERNACIONALES

path_c = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_d = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_r = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Evolución últimos 40 días de nuevos casos
def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Spain']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_italy(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Italy']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_france(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['France']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_germany(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Germany']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

'''Si se cogen los datos de España de John Hopkins
# data_spain_confirmed24h = create_dataframe(data_day, path_c, ['Confirmados (24h) en España'])
# data_spain_deaths24h = create_dataframe(data_day, path_d, ['Fallecidos (24h) en España'])
# data_spain_recovered24h = create_dataframe(data_day, path_r, ['Recuperados (24h) en España'])
#### Juntar dataframes
# data_spain_newcases24h = pd.concat([data_spain_confirmed24h,data_spain_recovered24h,data_spain_deaths24h], axis=0)
# data_spain_newcases24h.to_csv('data_spain_newcases24h.csv', sep = ';')
# data_spain_deaths24h.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')
'''

######## Lo mismo en italia
data_italy_confirmed24h = create_dataframe(data_day_italy, path_c, ['Confirmados (24h) en Italia'])
data_italy_deaths24h = create_dataframe(data_day_italy, path_d, ['Fallecidos (24h) en Italia'])
data_italy_recovered24h = create_dataframe(data_day_italy, path_r, ['Recuperados (24h) en Italia'])
data_italy_newcases24h = pd.concat([data_italy_confirmed24h,data_italy_recovered24h,data_italy_deaths24h], axis=0)

######## Lo mismo en Francia
data_france_confirmed24h = create_dataframe(data_day_france, path_c, ['Confirmados (24h) en Francia'])
data_france_deaths24h = create_dataframe(data_day_france, path_d, ['Fallecidos (24h) en Francia'])
data_france_recovered24h = create_dataframe(data_day_france, path_r, ['Recuperados (24h) en Francia'])
data_france_newcases24h = pd.concat([data_france_confirmed24h,data_france_recovered24h,data_france_deaths24h], axis=0)

######## Lo mismo en Alemania
data_germany_confirmed24h = create_dataframe(data_day_germany, path_c, ['Confirmados (24h) en Alemania'])
data_germany_deaths24h = create_dataframe(data_day_germany, path_d, ['Fallecidos (24h) en Alemania'])
data_germany_recovered24h = create_dataframe(data_day_germany, path_r, ['Recuperados (24h) en Alemania'])
data_germany_newcases24h = pd.concat([data_germany_confirmed24h,data_germany_recovered24h,data_germany_deaths24h], axis=0)


###### Juntar dataframes Spain + Italy sólo de fallecidos, OJO LOS DATOS DE SPAIN SE COGEN DEL CÁLCULO ANTERIOR DE DATADISTA
data_somecountries_newcases24h_deaths = pd.concat([data_spain_deaths24h_datadista, data_italy_deaths24h,
                                                   data_france_deaths24h, data_germany_deaths24h], axis=0)
data_somecountries_newcases24h_deaths.to_csv('data_somecountries_newcases24h.csv', sep = ';')
data_somecountries_newcases24h_deaths


,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,...,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12
Fallecidos (24h) en España,780,670,628,757,757,781,642,500,606,551,...,164,185,244,213,229,179,143,123,176,184
Fallecidos (24h) en Italia,766,681,525,636,604,542,610,570,619,431,...,174,195,236,369,274,243,194,165,179,172
Fallecidos (24h) en Francia,1122,1054,519,833,1417,544,1341,987,636,561,...,137,304,333,275,178,243,80,70,263,348
Fallecidos (24h) en Alemania,168,169,140,226,206,333,258,160,-31,286,...,54,127,0,282,117,118,39,20,92,77


## ESPAÑA: Evolución temporal casos confirmados

In [13]:
# Preparar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries.drop(columns=['cod_ine'])
data_ccaa_confirmed_timeseries_final.rename(columns = {'CCAA': 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.set_index('Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.iloc[0:19]

# Exportar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final.to_csv('data_ccaa_evolution_confirmed.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final

,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,...,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,0,0,0,0,0,0,1,6,8,12,...,14349,14394,14639,14870,15050,15228,15403,15478,15522,15629
Aragón,0,0,0,0,0,0,0,0,0,0,...,6239,6282,6382,6516,6575,6656,6702,6731,6797,6856
Asturias,0,0,0,0,0,0,0,0,0,1,...,3030,3058,3094,3099,3157,3183,3197,3212,3226,3252
Baleares,1,1,1,1,1,1,1,1,2,2,...,1991,1997,2013,2023,2029,2042,2055,2058,2058,2063
Canarias,1,1,1,1,2,5,6,6,6,7,...,2225,2231,2231,2235,2240,2250,2258,2260,2268,2271
Cantabria,0,0,0,0,0,0,0,0,0,1,...,2596,2607,2626,2642,2665,2681,2684,2685,2691,2733
Castilla La Mancha,0,0,0,0,0,0,0,0,0,1,...,21511,21656,22166,22567,22908,23293,23662,23869,24051,24185
Castilla y León,0,0,0,0,0,0,0,2,2,3,...,21379,21719,22156,22537,22926,23437,23786,23991,24170,24262
Cataluña,0,0,0,0,0,1,2,3,5,6,...,51393,51798,52090,52505,53209,53578,56520,56603,56787,56923


## ESPAÑA: Resúmenes por comunidades autónomas

In [14]:
# SOLO HACER CUANDO LOS DATOS EN DATADISTA ESTÉN ACTUALIZADOS, SI NO, NO FUNCIONA

# Confirmados por CCAA
data_ccaa_confirmed_today = data_ccaa_confirmed_timeseries[['CCAA', date_today_large]]
data_ccaa_confirmed_today.rename(columns = {date_today_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_today = data_ccaa_confirmed_today.set_index('Comunidades Autónomas')
data_ccaa_confirmed_today = data_ccaa_confirmed_today.copy().iloc[0:19]

# New cases
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_confirmed_daybefore.rename(columns = {date_daybefore_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_daybefore.set_index('Comunidades Autónomas')
data_ccaa_confirmed_newcases = data_ccaa_confirmed_today - data_ccaa_confirmed_daybefore
data_ccaa_confirmed_newcases = data_ccaa_confirmed_newcases.copy().iloc[0:19]
data_ccaa_confirmed_newcases.rename(columns = {'Confirmados': 'Confirmados (24h)'}, inplace = True)


# Fallecidos por CCAA
data_ccaa_deaths_today = data_ccaa_deaths_timeseries[['CCAA', date_today_large]]
data_ccaa_deaths_today.rename(columns = {date_today_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_today = data_ccaa_deaths_today.set_index('Comunidades Autónomas')
data_ccaa_deaths_today = data_ccaa_deaths_today.copy().iloc[0:19]

# New cases
data_ccaa_deaths_daybefore = data_ccaa_deaths_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_deaths_daybefore.rename(columns = {date_daybefore_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_daybefore = data_ccaa_deaths_daybefore.set_index('Comunidades Autónomas')
data_ccaa_deaths_newcases = data_ccaa_deaths_today - data_ccaa_deaths_daybefore
data_ccaa_deaths_newcases = data_ccaa_deaths_newcases.copy().iloc[0:19]
data_ccaa_deaths_newcases.rename(columns = {'Fallecidos': 'Fallecidos (24h)'}, inplace = True)


# Recuperados por CCAA
data_ccaa_recovered_today = data_ccaa_recovered_timeseries[['CCAA', date_today_large]]
data_ccaa_recovered_today.rename(columns = {date_today_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_today = data_ccaa_recovered_today.set_index('Comunidades Autónomas')
data_ccaa_recovered_today = data_ccaa_recovered_today.copy().iloc[0:19]
# New cases
data_ccaa_recovered_daybefore = data_ccaa_recovered_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_recovered_daybefore.rename(columns = {date_daybefore_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_daybefore = data_ccaa_recovered_daybefore.set_index('Comunidades Autónomas')
data_ccaa_recovered_newcases = data_ccaa_recovered_today - data_ccaa_recovered_daybefore
data_ccaa_recovered_newcases = data_ccaa_recovered_newcases.copy().iloc[0:19]
data_ccaa_recovered_newcases.rename(columns = {'Curados': 'Curados (24h)'}, inplace = True)


#Juntar los tres dataframes confirmados + curados + fallecidos
data_ccaa_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_today, data_ccaa_recovered_today, data_ccaa_deaths_today,], axis=1)

# Juntar los tres dataframes nuevos casos
data_ccaa_newcases_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_newcases, data_ccaa_recovered_newcases, data_ccaa_deaths_newcases], axis=1)

# Exportar por grupos
data_ccaa_confirmed_recovered_deaths.to_csv('data_ccaa_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_newcases_confirmed_recovered_deaths.to_csv('data_ccaa_newcases_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_ccaa_newcases_confirmed_recovered_deaths

,Confirmados (24h),Curados (24h),Fallecidos (24h)
Comunidades Autónomas,,,
Andalucía,107,184,6
Aragón,59,126,1
Asturias,26,7,3
Baleares,5,42,5
Canarias,3,19,0
Cantabria,42,73,2
Castilla La Mancha,134,24,24
Castilla y León,92,72,5
Cataluña,136,360,71


## ESPAÑA: Análisis por rangos de edad

In [15]:
#Comprobar si están subidos los últimos datos, si sale 0 es que no hay datos
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',')
data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large].count()

fecha                33
rango_edad           33
sexo                 33
casos_confirmados    33
hospitalizados       33
ingresos_uci         33
fallecidos           33
dtype: int64

In [16]:
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',', encoding = 'utf8', thousands = '.')
data_spain_ccaa_age = data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large]

def take_edad(sexo):
    data_spain_ccaa_age_all = data_spain_ccaa_age[data_spain_ccaa_age["sexo"] == sexo]
    #Para sacar datos total
    data_spain_ccaa_age_total = data_spain_ccaa_age_all.tail(1)
    data_spain_ccaa_age_total = data_spain_ccaa_age_total.iloc[:, [3,4,6]]
    data_spain_ccaa_age_total.columns = ['Confirmados','Hospitalizados', 'Fallecidos']
    #Para sacar datos concretos (de ambos, hombres y mujeres)
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_all[:-1]
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_concrete.iloc[:, [1,3,4,6]]
    data_spain_ccaa_age_concrete.columns = ['Rango de edad','Confirmados','Hospitalizados', 'Fallecidos']
    data_spain_ccaa_age_concrete.set_index('Rango de edad', inplace=True)
    return data_spain_ccaa_age_concrete, data_spain_ccaa_age_total


data_spain_ccaa_age_ambos_concrete, data_spain_ccaa_age_ambos_total = take_edad('ambos')
data_spain_ccaa_age_hombres_concrete, data_spain_ccaa_age_hombres_total = take_edad('hombres')
data_spain_ccaa_age_mujeres_concrete, data_spain_ccaa_age_mujeres_total = take_edad('mujeres')


data_spain_ccaa_age_ambos_total.index = ['Total ambos sexos']
data_spain_ccaa_age_hombres_total.index = ['Total hombres']
data_spain_ccaa_age_mujeres_total.index = ['Total mujeres']
data_spain_ccaa_age_ambos_concat = pd.concat([data_spain_ccaa_age_mujeres_total, data_spain_ccaa_age_hombres_total, 
                                             data_spain_ccaa_age_ambos_total], axis=0)

data_spain_ccaa_age_mujeres_concrete.to_csv('data_spain_ages_mujeres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_hombres_concrete.to_csv('data_spain_ages_hombres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_ambos_concat.to_csv('data_spain_ages_ambos.csv', sep = ';', encoding="utf-8")

print(data_spain_ccaa_age_ambos_concat)
print(data_spain_ccaa_age_hombres_concrete)

Confirmados  Hospitalizados  Fallecidos
Total mujeres           135047           40367        8066
Total hombres           102382           51340       10780
Total ambos sexos       237435           91710       18846
               Confirmados  Hospitalizados  Fallecidos
Rango de edad                                         
0-9                    465             151           1
10-19                  713             135           2
20-29                 4355             687          14
30-39                 8258            1928          39
40-49                14176            5038         129
50-59                18150            8307         431
60-69                17764           10491        1183
70-79                17854           12456        3108
80-89                15577            9718        4317
90 y +                5070            2429        1556


# Subir archivos a SERVIDOR desde FTP

In [17]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/
import csv
from ftplib import FTP
from io import StringIO

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]
ftp = FTP(host=host)
login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())

fp = open("data_world_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_resume.csv"), fp, 1024)

fp = open("data_world_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual.csv"), fp, 1024)


fp = open("data_world_actual_den.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual_den.csv"), fp, 1024)

fp = open("data_world_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_from100.csv"), fp, 1024)

fp = open("data_world_timeline.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_timeline.csv"), fp, 1024)

fp = open("data_somecountries_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_somecountries_newcases.csv"), fp, 1024)

fp = open("data_spain_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h.csv"), fp, 1024)

fp = open("data_spain_newcases24h_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h_deaths.csv"), fp, 1024)


# Datos por CCAA
fp = open("data_ccaa_evolution_confirmed.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_confirmed.csv"), fp, 1024)

fp = open("data_ccaa_evolution_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_deaths.csv"), fp, 1024)

fp = open("data_ccaa_evolution_recovered.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_recovered.csv"), fp, 1024)

fp = open("data_ccaa_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_confirmed_recovered_deaths.csv"), fp, 1024)

fp = open("data_ccaa_newcases_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_newcases_confirmed_recovered_deaths.csv"), fp, 1024)

# Datos por grupos de edad
fp = open("data_spain_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_mujeres.csv"), fp, 1024)

fp = open("data_spain_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_hombres.csv"), fp, 1024)

fp = open("data_spain_ages_ambos.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_ambos.csv"), fp, 1024)

fp.close()



230-OK. Current restricted directory is /
230 410 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 May  3 11:29 .
drwxr-xr-x    2 akakicre   akakicre         4096 May  3 11:29 ..
-rw-r--r--    1 akakicre   akakicre           10 May 12 18:45 .ftpquota
-rw-r--r--    1 akakicre   akakicre       154291 Apr  3 13:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          523 May 12 18:45 data_ccaa_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         7719 May 12 18:45 data_ccaa_evolution_confirmed.csv
-rw-r--r--    1 akakicre   akakicre         3603 May 12 18:45 data_ccaa_evolution_deaths.csv
-rw-r--r--    1 akakicre   akakicre         3064 May 12 18:45 data_ccaa_evolution_recovered.csv
-rw-r--r--    1 akakicre   akakicre          419 May 12 18:45 data_ccaa_newcases_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         1188 May 12 18:45 data_somecountries_newcases.csv
-rw-r--r--    1 akakicre   akakicre